## Batch fitting
This is the example ilustarting how fitting to multiple data sets can be acheived. This example has been developed during code CodeCampV in Grenoble, Apr 2017 by Miguel Gonzales and Marco Adamo.
The script reads a series of files from GRASP and fits the model defined by model.py.
For example ellipsoid.py fits a ellipsoid form factor model.
For each file a directory named Fit_filename is created. The file model.par contains the fitted parameters, which are gathered together into batch_fit.csv in the current directory.
Finally the fitted parameters are plotted for the full series.

In [1]:
import sys
import os
import numpy as np

import matplotlib.pyplot as plt
from bumps.dream.views import tile_axes  # make a grid of plots

### Load data and set up parameters

In [5]:
path = 'notebook_files'
model_file = f"{path}/custom_ellipsoid.py"
data_files = ["093191_201.dat", "093192_202.dat","093193_203.dat", "093194_204.dat", "093195_205.dat"]
#ftol: f(x) tolerance uses differences in the function value to decide when the fit is complete
#xtol: x tolerance uses differences in the parameter value to decide when the fit is complete
fit_opts = "--fit=lm --steps=200 --ftol=1.5e-8 --xtol=1.5e-8"
fit_opts = [v for v in fit_opts if v.startswith('--')]

### Check that data files exist

In [6]:
def fit_dir(filename):
    "Return the store directory name for the given file"
    return "Fit_" + os.path.splitext(filename)[0]

### Loop over files in bumps
**Note:** Bumps is run from command line

In [7]:
bumps_cmd = "python -m bumps.cli --batch"
fit_opts = " ".join(fit_opts)
for data_file in data_files:
    data_file = f"{path}/{data_file}"
    store_opts = "--store=" + fit_dir(data_file)
    cmd = " ".join((bumps_cmd, fit_opts, store_opts, model_file, data_file))
    os.system(cmd)

Please check your file path and try again.
Traceback (most recent call last):
  File "/Users/wojciechpotrzebowski/opt/anaconda3/envs/sasview_devt/lib/python3.9/site-packages/sasview-5.0.5-py3.9.egg/sas/sascalc/dataloader/loader.py", line 74, in load
    data_list = super(Registry, self).load(path, format=format)
  File "/Users/wojciechpotrzebowski/opt/anaconda3/envs/sasview_devt/lib/python3.9/site-packages/sasview-5.0.5-py3.9.egg/sas/sascalc/data_util/registry.py", line 142, in load
    raise last_exc
  File "/Users/wojciechpotrzebowski/opt/anaconda3/envs/sasview_devt/lib/python3.9/site-packages/sasview-5.0.5-py3.9.egg/sas/sascalc/data_util/registry.py", line 137, in load
    return fn(path)
  File "/Users/wojciechpotrzebowski/opt/anaconda3/envs/sasview_devt/lib/python3.9/site-packages/sasview-5.0.5-py3.9.egg/sas/sascalc/dataloader/file_reader_base_class.py", line 118, in read
    self.handle_error_message(msg)
  File "/Users/wojciechpotrzebowski/opt/anaconda3/envs/sasview_devt/lib/pyt

### Gather results

In [ ]:
results = {}
par_file = os.path.splitext(model_file)[0] + '.par'
for data_file in data_files:
    with open(os.path.join(fit_dir(data_file), par_file), 'r') as fid:
        for line in fid:
            parameter, value = line.split()
            results.setdefault(parameter, []).append(float(value))

### Save results into file

In [ ]:
with open('batch_fit.csv', 'w') as fid:
    parameters = list(sorted(results.keys()))
    values_by_file = zip(*(v for k, v in sorted(results.items())))
    fid.write(','.join(['filename'] + parameters) + '\n')
    for filename, values in zip(data_files, values_by_file):
        fid.write(','.join([filename] + [str(v) for v in values]) + '\n')

### Show fitted parameters

In [ ]:
nh, nw = tile_axes(len(results))
ticks = np.arange(1, len(data_files)+1)
labels = [os.path.splitext(filename)[0] for filename in data_files]
for k, (parameter, values) in enumerate(sorted(results.items())):
    plt.subplot(nh, nw, k+1)
    plt.plot(ticks, values)
    plt.xlim(ticks[0]-0.5, ticks[-1]+0.5)
    if k%nh == nh-1:
        plt.xticks(ticks, labels, rotation=30)
    else:
        plt.xticks(ticks, [' ']*len(labels))
    plt.ylabel(parameter)
plt.suptitle("Fits")
plt.show()